In [8]:
import numpy as np
import pandas as pd
from scipy.interpolate import Rbf

def benchmark(x1, x2):
    return (    (1.3356 * (1.5 * (1 - x1))) 
                + (np.exp((2 * x1) - 1) * np.sin((3 * np.pi) * ((x1 - 0.6) ** 2)))
                + (np.exp(3 * (x2 - 0.5)) * np.sin((4 * np.pi) * ((x2 - 0.9) ** 2)))
            )

def getData(grid):
    x1 = np.linspace(0, 1, grid)
    x2 = np.linspace(0, 1, grid)
    x1, x2 = np.meshgrid(x1, x2)
    f_x1_x2 = benchmark(x1, x2)
    data = {'x1': x1.flatten(), 'x2': x2.flatten(), 'f(x1,x2)': f_x1_x2.flatten()}
    df = pd.DataFrame(data)
    return df

def Kriging(x_orig, y_orig, z_orig,):
    kriging = Rbf(x_orig, y_orig, z_orig, function='gaussian')
    x_new = np.linspace(min(x_orig), max(x_orig), 30)
    y_new = np.linspace(min(y_orig), max(y_orig), 30)
    X_new, Y_new = np.meshgrid(x_new, y_new)
    Z_new = kriging(X_new, Y_new)

    indices = np.random.choice(X_new.shape[0]*X_new.shape[1], size=135, replace=False)
    x_virtual = X_new.flatten()[indices]
    y_virtual = Y_new.flatten()[indices]
    z_virtual = Z_new.flatten()[indices]

    return(x_virtual, y_virtual, z_virtual) 

df_1000 = getData(32)
df_test = pd.read_excel("Kriging-data.xlsx", sheet_name="Test")
df_training = pd.read_excel("Kriging-data.xlsx", sheet_name="Training")

In [9]:
import plotly.figure_factory as ff
import plotly.offline as py
from scipy.spatial import Delaunay
import plotly.graph_objs as go

def PlotSurface():
        x_sup, y_sup, z_sup = df_1000['x1'], df_1000['x2'], df_1000['f(x1,x2)']
        x_orig, y_orig, z_orig = df_training['x1'], df_training['x2'], df_training['f(x1,x2)']
        x_virtual, y_virtual, z_virtual = Kriging(x_orig, y_orig, z_orig)
        points = np.column_stack((x_sup, y_sup))
        tri = Delaunay(points)
        
        fig_surface = ff.create_trisurf(x=x_sup, y=y_sup, z=z_sup, 
                                        simplices=tri.simplices, colormap='Viridis',
                                        title='Surface Fit - Original Values')

        fig_orig_data =  go.Scatter3d(
                x=x_orig,
                y=y_orig,
                z=z_orig,
                mode='markers',
                marker=dict(
                size=5,
                color='red',  
                opacity=0.8
                ),
                name='Original Samples'
        )

        fig_virtual_data = go.Scatter3d(
        x=x_virtual,
        y=y_virtual,
        z=z_virtual,
        mode='markers',
        marker=dict(
            size=4,
            color='blue',  
            opacity=0.8
        ),
        name='Virtual Samples'
    )

        data = [fig_surface.data[0], fig_orig_data, fig_virtual_data]
        layout = go.Layout(
        title='Surface Fit',
        scene=dict(
            xaxis=dict(title='X1'),
            yaxis=dict(title='X2'),
            zaxis=dict(title='f(x1, x2)')
        ),
        width=500,
        height=500,
        margin=dict(l=65, r=50, b=65, t=90),
    )

        fig = go.Figure(data=data, layout=layout)
        py.iplot(fig)

PlotSurface()